In [4]:
from kaggle_environments import make, evaluate

All strategies are tested against the random action choice during each round of the game:

(1) Random Goblin

In [194]:
%%writefile random_goblin.py
import random
def random_goblin(observation, configuration):
    ''' each round - random action '''
    return random.randrange(0, configuration.signs)

Overwriting random_goblin.py


In [196]:
evaluate(
    "rps", 
    ["random_goblin.py", "random_goblin.py"],
    configuration={"episodeSteps": 500}
)

[[20.0, -20.0]]

(2) Scissors only:

In [197]:
%%writefile scissors_only.py
def scissors_only(observation, configuration):
    ''' the agent applies scissors action every round '''
    return 2

Writing scissors_only.py


In [198]:
evaluate(
    "rps", 
    ["scissors_only.py", "random_goblin.py"],
    configuration={"episodeSteps": 500}
)

[[0, 0]]

(3) Paper only:

In [199]:
%%writefile paper_only.py
def paper_only(observation, configuration):
    ''' the agent applies paper action every round '''
    return 1

Writing paper_only.py


In [200]:
evaluate(
    "rps", 
    ["paper_only.py", "random_goblin.py"],
    configuration={"episodeSteps": 500}
)

[[-56.0, 56.0]]

(4) Rock only

In [202]:
%%writefile rock_only.py
def rock_only(observation, configuration):
    ''' the agent applies rock action every round '''
    return 0

Overwriting rock_only.py


In [203]:
evaluate(
    "rps", 
    ["rock_only.py", "random_goblin.py"],
    configuration={"episodeSteps": 500}
)

[[0, 0]]

(5) sequential bot

In [204]:
%%writefile sequential.py
import random
def sequential(observation, configuration):
    ''' the agent randomly applies first action
    then sequentially selects the next action '''
    global my_action # variable to store the agent's own action
    if observation.step == 0: # first action is random
        my_action = random.randrange(0, configuration.signs)
    else:
        # action cycling from 0 up to 2
        my_action = my_action+1 if my_action < 2 else 0
        
    return my_action

Writing sequential.py


In [205]:
evaluate(
    "rps", 
    ["sequential.py", "random_goblin.py"],
    configuration={"episodeSteps": 500}
)

[[36.0, -36.0]]

(6) copycat 

In [207]:
%%writefile copycat.py
import random
def copycat(observation, configuration):
    ''' repeats the opponent's latest action '''  
    return observation.lastOpponentAction if observation.step != 0 else random.randrange(0, configuration.signs) 

Overwriting copycat.py


In [209]:
evaluate(
    "rps", 
    ["copycat.py", "random_goblin.py"],
    configuration={"episodeSteps": 500}
)

[[36.0, -36.0]]

(7) bot with late reaction

In [210]:
%%writefile late_reaction.py
import random
def late_reaction(observation, configuration):
    ''' reacting to the opponent's latest action in a new round '''
    if observation.step == 0: # first action is random
        my_action = random.randrange(0, configuration.signs)
    else:
        my_action = observation.lastOpponentAction + 1 if observation.lastOpponentAction < 2 else 0
        
    return my_action

Overwriting late_reaction.py


In [211]:
evaluate(
    "rps", 
    ["late_reaction.py", "random_goblin.py"],
    configuration={"episodeSteps": 500}
)

[[0, 0]]

(8) bot that assumes the opponent never repeats the same action twice in a row

In [212]:
%%writefile never_twice.py
import random
def never_twice(observation, configuration):
    ''' the agent assumes the opponent won't repeat his action in the next round 
    thus the agent applies randomly the action againt the leftovers of the opponent'''
    if observation.step == 0: # first action is random
        my_action = random.randrange(0, configuration.signs)
    else:
        # exclude the previous opponent's action
        leftovers = [x for x in [0, 1, 2] if x != observation.lastOpponentAction]
        # random selection (guess) of the opponent's next move
        op_action = random.choice(leftovers)
        # make better action
        my_action = op_action + 1 if op_action < 2 else 0
        
    return my_action

Overwriting never_twice.py


In [213]:
evaluate(
    "rps", 
    ["never_twice.py", "random_goblin.py"],
    configuration={"episodeSteps": 500}
)

[[-20.0, 20.0]]

(9) bot that counters the most frequent opponent's action

In [214]:
%%writefile counting_bot.py
import random
history = {0: 0, 1: 0, 2: 0} # opponent's moves counter
def counting_bot(observation, configuration):
    ''' counts the opponents previous moves and selects most frequent '''
    global history
    if observation.step == 0: # first action is random
        my_action = random.randrange(0, configuration.signs)
    else:
        # count the opponents move
        history[observation.lastOpponentAction] += 1
        # get the most frequent opponent's action 
        op_action = max(history, key=history.get)
        # counter move
        my_action = op_action + 1 if op_action < 2 else 0
        
    return my_action

Overwriting counting_bot.py


In [215]:
evaluate(
    "rps", 
    ["never_twice.py", "random_goblin.py"],
    configuration={"episodeSteps": 500}
)

[[0, 0]]

(10) bot with ML engine (ensemble)

In [180]:
%%writefile ensemble_bot.py
import math
import random
import numpy as np
from sklearn.ensemble import RandomForestRegressor

def get_score(left_move, right_move):
    ''' get the score of a round '''
    delta = (
        right_move - left_move
        if (left_move + right_move) % 2 == 0
        else left_move - right_move
    )
    return 0 if delta == 0 else math.copysign(1, delta)

# statistic matrix n x (my_action, op_action, score)
history = {}

def ensemble_bot(observation, configuration):
    ''' applies ensemble ML engine to predict the next move 
    the features are moves, the target is score'''
    global history 
    
    # all possible combos
    all_combos = np.array([
        [0, 0], [0, 1], [0, 2],
        [1, 0], [1, 1], [1, 2],
        [2, 0], [2, 1], [2, 2]
    ])
    
    # first 10 rounds are recorded for the train set
    if observation.step == 0:
        my_action = random.randrange(0, configuration.signs)
        op_action = -1
        history = np.array([
            my_action, op_action, get_score(my_action, op_action)
        ])
    elif 0 < observation.step <= 11:
        my_action = random.randrange(0, configuration.signs)
        op_action = observation.lastOpponentAction
        history = np.vstack([
            history,
            np.array([my_action, op_action, get_score(my_action, op_action)])
        ])
    else:
        # the zero observation dropped
        history = history[1:,:]
        # for the target the ties and losses are 0, wins are 1
        history[:, -1][history[:, -1] == -1] = 0
        # the engine is initialized and fit every round
        mind = RandomForestRegressor()
        mind.fit(history[:,:-1], history[:,-1])
        # collecting the probs of target == 1 (win) for all possible combos
        all_probs = [mind.predict([combo])[0] for combo in all_combos]
        # my action is the action with the highest win probability
        my_action = int(all_combos[np.argmax(all_probs)][0])
        # the round is added to the train dataset
        op_action = observation.lastOpponentAction
        history = np.vstack([
            history,
            np.array([my_action, op_action, get_score(my_action, op_action)])
        ])

    return my_action


Overwriting ensemble_bot.py


In [181]:
env = make("rps", debug=True, configuration={"episodeSteps":500})
env.run(["random_goblin.py", "ensemble_bot.py"])
env.render(mode="ipython", width=500, height=450)

In [184]:
evaluate(
    "rps", 
    ["ensemble_bot.py", "random_goblin.py"],
    configuration={"episodeSteps": 500}
)

[[0, 0]]

(11) bot with ML engine (linear)

In [190]:
%%writefile linear_bot.py
import math
import random
import numpy as np
from sklearn.linear_model import LinearRegression

def get_score(left_move, right_move):
    ''' get the score of a round '''
    delta = (
        right_move - left_move
        if (left_move + right_move) % 2 == 0
        else left_move - right_move
    )
    return 0 if delta == 0 else math.copysign(1, delta)

# statistic matrix n x (my_action, op_action, score)
history = {}

def ensemble_bot(observation, configuration):
    ''' applies linear ML engine to predict the next move 
    the features are moves, the target is score'''
    global history 
    
    # all possible combos
    all_combos = np.array([
        [0, 0], [0, 1], [0, 2],
        [1, 0], [1, 1], [1, 2],
        [2, 0], [2, 1], [2, 2]
    ])
    
    # first 10 rounds are recorded for the train set
    if observation.step == 0:
        my_action = random.randrange(0, configuration.signs)
        op_action = -1
        history = np.array([
            my_action, op_action, get_score(my_action, op_action)
        ])
    elif 0 < observation.step <= 11:
        my_action = random.randrange(0, configuration.signs)
        op_action = observation.lastOpponentAction
        history = np.vstack([
            history,
            np.array([my_action, op_action, get_score(my_action, op_action)])
        ])
    else:
        # the zero observation dropped
        history = history[1:,:]
        # for the target the ties and losses are 0, wins are 1
        history[:, -1][history[:, -1] == -1] = 0
        # the engine is initialized and fit every round
        mind = LinearRegression()
        mind.fit(history[:,:-1], history[:,-1])
        # collecting the logits (~ probs) of target == 1 (win) for all possible combos
        all_probs = [mind.predict([combo])[0] for combo in all_combos]
        # my action is the action with the highest win probability
        my_action = int(all_combos[np.argmax(all_probs)][0])
        # the round is added to the train dataset
        op_action = observation.lastOpponentAction
        history = np.vstack([
            history,
            np.array([my_action, op_action, get_score(my_action, op_action)])
        ])

    return my_action


Overwriting linear_bot.py


In [191]:
env = make("rps", debug=True, configuration={"episodeSteps":500})
env.run(["linear_bot.py", "random_goblin.py"])
env.render(mode="ipython", width=500, height=450)

In [192]:
evaluate(
    "rps", 
    ["linear_bot.py", "random_goblin.py"],
    configuration={"episodeSteps": 500}
)

[[34.0, -34.0]]

(12) bot with ML engine (neural perceptron)

In [223]:
%%writefile perceptron.py
import math
import random
import numpy as np
from sklearn.neural_network import MLPRegressor
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

def get_score(left_move, right_move):
    ''' get the score of a round '''
    delta = (
        right_move - left_move
        if (left_move + right_move) % 2 == 0
        else left_move - right_move
    )
    return 0 if delta == 0 else math.copysign(1, delta)

# statistic matrix n x (my_action, op_action, score)
history = {}

def ensemble_bot(observation, configuration):
    ''' applies linear ML engine to predict the next move 
    the features are moves, the target is score'''
    global history 
    
    # all possible combos
    all_combos = np.array([
        [0, 0], [0, 1], [0, 2],
        [1, 0], [1, 1], [1, 2],
        [2, 0], [2, 1], [2, 2]
    ])
    
    # first 10 rounds are recorded for the train set
    if observation.step == 0:
        my_action = random.randrange(0, configuration.signs)
        op_action = -1
        history = np.array([
            my_action, op_action, get_score(my_action, op_action)
        ])
    elif 0 < observation.step <= 11:
        my_action = random.randrange(0, configuration.signs)
        op_action = observation.lastOpponentAction
        history = np.vstack([
            history,
            np.array([my_action, op_action, get_score(my_action, op_action)])
        ])
    else:
        # the zero observation dropped
        history = history[1:,:]
        # for the target the ties and losses are 0, wins are 1
        history[:, -1][history[:, -1] == -1] = 0
        # the engine is initialized and fit every round
        mind = MLPRegressor()
        mind.fit(history[:,:-1], history[:,-1])
        # collecting the logits (~ probs) of target == 1 (win) for all possible combos
        all_probs = [mind.predict([combo])[0] for combo in all_combos]
        # my action is the action with the highest win probability
        my_action = int(all_combos[np.argmax(all_probs)][0])
        # the round is added to the train dataset
        op_action = observation.lastOpponentAction
        history = np.vstack([
            history,
            np.array([my_action, op_action, get_score(my_action, op_action)])
        ])

    return my_action


Overwriting perceptron.py


In [224]:
env = make("rps", debug=True, configuration={"episodeSteps":500})
env.run(["perceptron.py", "random_goblin.py"])
env.render(mode="ipython", width=500, height=450)

In [225]:
evaluate(
    "rps", 
    ["perceptron.py", "random_goblin.py"],
    configuration={"episodeSteps": 500}
)

[[-25.0, 25.0]]